## Mixing Sage and Amber force fields: BRD4 benchmark

This example applies SMIRNOFF-format parameters to BRD4 inhibitors from the [living review on binding free energy benchmark systems](https://www.annualreviews.org/doi/abs/10.1146/annurev-biophys-070816-033654) by Mobley and Gilson. The BRD4 system comes from the [accompanying GitHub repository](https://github.com/MobleyLab/benchmarksets/tree/master/input_files/BRD4).

In [1]:
# Retrieve protein and ligand files for BRD4 and a docked inhibitor from the benchmark systems GitHub repository
# https://github.com/MobleyLab/benchmarksets
import requests

repo_url = (
    "https://raw.githubusercontent.com/MobleyLab/benchmarksets/master/input_files/"
)

sources = {
    "receptor.pdb": f"{repo_url}BRD4/pdb/BRD4.pdb",
    "ligand.pdb": f"{repo_url}BRD4/pdb/ligand-1.pdb",
    "ligand.sdf": f"{repo_url}BRD4/sdf/ligand-1.sdf",
}
for filename, url in sources.items():
    r = requests.get(url)
    open(filename, "w").write(r.text)

In [2]:
from openff.toolkit import ForceField, Molecule, Quantity, Topology

In [3]:
ligand_molecule = Molecule.from_file("ligand.sdf")
sage = ForceField("openff-2.2.0.offxml")

ligand = sage.create_interchange(topology=ligand_molecule.to_topology())

receptor_topology = Topology.from_pdb("receptor.pdb")

ff14sb = ForceField("ff14sb_off_impropers_0.0.3.offxml")

receptor = ff14sb.create_interchange(topology=receptor_topology)

# Interchange.combine works nicely for some cases but hasn't been tested for all - use caution
# if adapting this for more novel workflows!
complex_system = receptor.combine(ligand)

/Users/mattthompson/micromamba/envs/openff-toolkit-test/lib/python3.10/site-packages/openff/interchange/operations/_combine.py:52: InterchangeCombinationWarning: Interchange object combination is complex and likely to produce strange results outside of a limited set of use cases it has been tested in. Any workflow using this method is not guaranteed to be suitable for production or stable between versions. Use with extreme caution and thoroughly validate results!
  warnings.warn(


### Export to OpenMM

See [docs](https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html#openmm) for more options.

In [4]:
complex_system.to_openmm()

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x14ba376f0> >

### Export to Amber

See [docs](https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html#amber) for more options.

In [5]:
complex_system.to_inpcrd("complex.inpcrd")
complex_system.to_prmtop("complex.prmtop")

### Export to GROMACS

See [docs](https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html#gromacs) for more options.

GROMACS does not support vacuum simulations. In this example, we didn't add solvent and the original PDB file didn't have box vectors. We'll just arbitrarily add a 4 nm cubic box around the receptor-ligand complex.

In [6]:
complex_system.box = Quantity([4, 4, 4], "nanometer")

# This step might be slow, writing some large proteins is not yet optimized
complex_system.to_gromacs("complex")

/Users/mattthompson/micromamba/envs/openff-toolkit-test/lib/python3.10/site-packages/openff/interchange/components/mdconfig.py:471: UserWarning: Ambiguous failure while processing constraints. Constraining h-bonds as a stopgap.
  warnings.warn(
